In [1]:
import requests

In [2]:
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import re
from geopy.geocoders import Nominatim
import json
import datetime as dt

In [4]:
geolocator = Nominatim(user_agent="extractor")

Scraping

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="extractor")

def replaceMultiple(string, replace=[], replaceWith=''):
    for item in replace:
        string = string.replace(item, replaceWith)
    return string
abbrevated_places = {'UK':'Uttrakhand','UP':"Uttar Pradesh",'uttar-pradesh':"Uttar Pradesh",'MP':"Madhya Pradesh","madhya-pradesh":"Madhya Pradesh"}
def addData(data, place, link, title, image):
    data['count']+=1
    if(place not in data['data']):
        if(len(place) == 2):
            if(place.upper() not in title):
                return
            place = abbrevated_places[place]
        elif('-' in place):
            place = abbrevated_places[place]
        cord = geolocator.geocode(place)
        data['data'][place] = {'cordinates' : [cord.longitude, cord.latitude],'count': 0 , 'links': [{'title':title, 'url':link, 'img':image}]}
    else:
        if(len(place) == 2):
            if(place.upper() not in title):
                return
            place = abbrevated_places[place]
        elif('-' in place):
            place = abbrevated_places[place]
        data['data'][place]['links'].append({'title':title, 'url':link, 'img':image})
    data['data'][place]['count'] = len(data['data'][place]['links'])

    return data

List of cities, States, UTs

In [6]:
# from bs4 import BeautifulSoup as bs
# import requests
# import re
import json

# s = requests.get('https://www.krisia.com/city_list.htm')
# ne = bs(s.content, 'html.parser').select('td.resultsPane > a')
# places = []
# for it in ne:
#     if(len(it.text)>1 and it.text!="Top"):
#         st = []
#         for w in it.text.split(" "):
#             st.append(w[0]+ w[1:].lower())
#         places.append(" ".join(st))

# aa = """Andhra Pradesh
# Arunachal Pradesh
# Assam
# Bihar
# Chhattisgarh
# Goa
# Gujarat
# Haryana
# Himachal Pradesh
# Jharkhand
# Karnataka
# Kerala
# Madhya Pradesh
# Maharashtra
# Manipur
# Meghalaya
# Mizoram
# Nagaland
# Odisha
# Punjab
# Rajasthan
# Sikkim
# Tamil Nadu
# Telangana
# Tripura
# Uttarakhand
# Uttar Pradesh
# West Bengal"""

# #(Added UTs)
# places.extend(aa.split('\n'))
# places.extend(["Bengaluru","Palghar","Andaman","Nicobar","Daman","Diu","Dadra","Nagara","Puducherry","Jammu","Kashmir","Lakshwadeep"])
# places.extend(["Bubaneswar"])
# places.remove("New Delhi")
# with open('./places.json', 'w') as jso:
#     json.dump(places,jso,indent=3)
places = json.load(open('../data/places.json'))


Zee news

In [ ]:
def zee():
    data = {}
    data['source_tag'] = 'zee'
    data['name'] = 'Zee news'
    data['image'] = 'https://www.timesnownews.com/assets/icons/svg/times-now.svg'
    data['count'] = 0
    data['data'] = {}
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
    for a in range(0,10):
        print("Scraping page",a)
        articles = bs(requests.get('https://zeenews.india.com/common/getmorenews/%x/120183'%(a),headers=agent).content,'html.parser').select('div.row')
        for news in articles:
            image = news.find('img')['src']
            link = 'https://zeenews.india.com/' + news.find('a')['href']
            title = news.find('a').text
            time = bs(requests.get(link,headers=agent).content,'html.parser').select('div.articleauthor_details > span')[-4].text[:-4]
            if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time, "%b %d, %Y, %I:%M %p")):
                return data
            for place in places:
                if(re.search(place ,link,re.IGNORECASE)):
                    addData(data, place, link, title, image)
                    break
zee()

Times Now

In [29]:
errors = []
def timnow():
    data = {}
    data['source_tag'] = 'timnow'
    data['name'] = 'Times Now'
    data['image'] = 'https://www.timesnownews.com/assets/icons/svg/times-now.svg'
    data['count'] = 0
    data['data'] = {}

    articles = json.loads(requests.get('https://apiprod.timesnownews.com/api/getlisting?seopath=india&pageno=1&itemcount=50').content)['response']['sections']["tnn_87844847"]['children']
    for article in articles:
        try:
            title = article['title']
            link = 'https://www.timesnownews.com/' + article['seopath'] + '-article-' + str(article['msid'])
            time = article['metainfo']['LastPublishMilliTime']['value']
            image  = 'https://static.tnn.in/photo/msid-{msid},imgsize-26308,updatedat-{upd},width-200,height-200,resizemode-75/{msid}.jpg'.format(msid = article['msid'], upd= article['updatedate'])
            #now - 1 day > start + millisec === now > start + 1 + millisec
            print(title)
            if(dt.datetime.now()>dt.datetime(year=1970,month=1,day=2,hour=5,minute=30,second=1)+dt.timedelta(milliseconds=int(time))):
                return data
            for place in places:
                if(re.search("[\/-]"+place+"[\/-]",link,re.IGNORECASE)):
                    addData(data, place, link, title, image)
        except Exception as e:
                errors.append((data['source_tag'],link,e))
    return data

timnow()

Tamil Nadu: Man loses tongue due to snake bite; thanks to astrologer’s advice
'Judges must speak through judgement': Law Min Kiren Rijiju's bold remarks at Times Now Summit 2022
Govt school students in rural Jharkhand get wings for their JEE and NEET aspirations
Maharashtra: 4 girls die after drowning in Kitwad waterfall while trying to click photos; another critical
Security forces kill four Maoist militants in encounter in Chhattisgarh
'PM Modi has been very firm': EAM Jaishankar's response to opposition on China dispute at Times Now Summit 2022
Times Now Summit 2022: Voices that matter participate in a thought-provoking dialogue - Who said what
Opposing Hindi 'imposition', 85-year-old man sets himself on fire in Tamil Nadu's Salem, dies
WATCH: ISRO successfully launches PSLV C-54 rocket carrying Oceansat, customer nanosatellites
BJP president Nadda releases manifesto for Gujarat assembly polls; Key promises
Rajasthan health infrastructure in ICU: Patient dies as people push fuel-les

{'source_tag': 'timnow',
 'name': 'Times Now',
 'image': 'https://www.timesnownews.com/assets/icons/svg/times-now.svg',
 'count': 18,
 'data': {'Jharkhand': {'cordinates': [85.2557301, 23.4559809],
   'count': 1,
   'links': [{'title': 'Govt school students in rural Jharkhand get wings for their JEE and NEET aspirations',
     'url': 'https://www.timesnownews.com/india/govt-school-students-in-rural-jharkhand-get-wings-to-their-jee-and-neet-aspirations-article-95786580',
     'img': 'https://static.tnn.in/photo/msid-95786580,imgsize-26308,updatedat-1669459951041,width-200,height-200,resizemode-75/95786580.jpg'}]},
  'Maharashtra': {'cordinates': [75.6741579, 18.9068356],
   'count': 1,
   'links': [{'title': 'Maharashtra: 4 girls die after drowning in Kitwad waterfall while trying to click photos; another critical',
     'url': 'https://www.timesnownews.com/india/maharashtra-4-girls-die-after-drowning-in-kitwad-waterfall-while-trying-to-click-photos-another-critical-article-95785490',
 

In [105]:
'UK' in 'uk'

False

In [ ]:
def et():
    data = {}
    data['source_tag'] = 'et'
    data['name'] = 'The Economic Times'
    data['image'] = ''
    data['count'] = 0
    data['data'] = {}
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

    for a in range(1,10):
        print('Scraping page ',a)
        document = bs(requests.get('https://economictimes.indiatimes.com/lazyloadlistnew.cms?msid=81582957&curpg='+str(a),headers = agent).content,'html.parser')
        for article in document.select('div.eachStory'):
            image = 'https://economictimes.indiatimes.com/' + article.select_one('img')['data-original']
            link = 'https://economictimes.indiatimes.com/' + article.select_one('a')['href']
            title = article.select_one('h3>a').text
            time = article.select_one('time')['data-time'][:-4]
            if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time, "%b %d, %Y, %I:%M %p")):
                return data
            for place in places:
                if(re.search(place ,link,re.IGNORECASE)):
                    addData(data, place, link, title, image)
                    break
    return data
print(et())

CNBC

In [ ]:
def cnbc():
    data = {}
    data['source_tag'] = 'cnbc'
    data['name'] = 'CNBC TV18'
    data['image'] = 'https://upload.wikimedia.org/wikipedia/commons/2/23/CNBC_TV18_logo.png'
    data['count'] = 0
    data['data'] = {}

    news_info = json.loads(requests.get('https://www.cnbctv18.com/api/v1/category/india?page=1&limit=40').content)['result']
    for news in news_info:
        print(news,end="\n\n")
        title = news['headline']
        time = news['update_date']
        link = 'https://www.cnbctv18.com/' + news['posturl']
        image = list(json.loads(news['post_image_arr'])['images'].values())[0]
        if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(str(time), "%Y%m%d%H%M%S")):
            return data
        for place in places:
            if(re.search(place ,link,re.IGNORECASE)):
                addData(data, place, link, title, image)
                break
    return data
print(cnbc())

News18

In [ ]:
def news18():
    data = {}
    data['source_tag'] = 'news18'
    data['name'] = 'News 18'
    data['image'] = 'https://www.adgully.com/img/800/201906/news-18-india-2.jpg'
    data['count'] = 0
    data['data'] = {}

    for a in range(1,5):
        print('Scraping page ',a)
        articles = bs(requests.get('https://www.news18.com/india/page-' + str(a)).content, 'html.parser').select('div.blog_list_row')
        for article in articles:
            link = article.select_one('a')['href']
            title = article.select_one('div.blog_title').text
            document = bs(requests.get(link).content, 'html.parser')
            loc = document.find('span',{"id": "location_info"})
            if document.select_one('figure > img'):
                time = document.select('div.article_details_list > div')[-2].text[14:-4]
                if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time, "%B %d, %Y, %H:%M")):
                    return data
                image = document.select_one('figure > img')['src']
                for place in places:
                    if(re.search(place ,link,re.IGNORECASE)):
                        addData(data, place, link, title, image)
                        break
    return data
news18()

Hindustan Times

In [ ]:
def ht():
    data = {}
    data['source_tag'] = 'ht'
    data['name'] = 'Hindustan Times'
    data['image'] = 'https://www.hindustantimes.com/res/images/logo.png'
    data['count']=0
    data['data'] = {}
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

    for a in range(1,10):
        print("scraping page " , a)
        document = requests.get('https://www.hindustantimes.com/cities/page-' + str(a),headers=agent)
        news_divs = bs(document.content, 'html.parser').select("div[data-vars-storytype='story']")
        for news in news_divs:
            title = news.select_one('h3.hdg3 > a').text
            #upload time
            ti = replaceMultiple(news.find("div", class_= "dateTime").text, ["IST", "Published", "Updated", "on",","]).split(" ")[2:7]
            #ti -> [Mon, DD, YYYY, HH:MM, AM/PM]
            if(dt.datetime.now()-dt.timedelta(hours=18,minutes=30)>dt.datetime.strptime(" ".join(ti), "%b %d %Y %I:%M %p")):
                return data
                #break if article was posted more than 24 hours ago
            link = 'https://www.hindustantimes.com' + news.select_one('h3.hdg3 > a')['href']
            image = bs(requests.get(link,headers=agent).content, 'html.parser').select_one('div.fullStory').find("img")['src']
            for place in places:
                if(re.search("[\/-]"+place+"[\/-]",link,re.IGNORECASE)):
                    addData(data, place, link, title, image)
                    data['count']+=1
                
    return data
ht()

Indian Express

In [ ]:
def abp():
    data = {}
    data['source_tag'] = 'abp'
    data['name'] = 'ABP News'
    data['image'] = 'https://static.abplive.com/frontend/images/nw-eng-og.png?impolicy=abp_cdn&imwidth=600'
    data['data'] = {}

    for a in range(1,10):
        print('Scraping page ',a)

        document = requests.get('https://news.abplive.com/news/india/page-' + str(a))
        articles = bs(document.content, 'html.parser').select('div.uk-width-3-4 > div > div > div.other_news')[:19]

        for article in articles:
            link = article.find('a')['href']
            title = article.find('a')['title']
            image = article.find('img')['data-src']
            doc_cont = bs(requests.get(link).content, 'html.parser').select_one('p.article-author').text[5:]
            time = doc_cont[doc_cont.index(':')+2:doc_cont.index('(')-1]
            if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time, "%d %b %Y %I:%M %p")):
                return data
            for place in places:
                if(re.search(place ,link,re.IGNORECASE)):
                    addData(data, place, link, title, image)
                    break

ABP

In [9]:
errors=[]

def inexp():
    data = {}
    data['source_tag'] = 'inexp'
    data['name'] = 'The Indian Express'
    data['image'] = 'https://play-lh.googleusercontent.com/dSS5OclMxGTasbTH1PYsxZ9bmXZyv7xcU4elR7afSqXns-6MEo1ZYteZi-l75E3g5kY'
    data['count'] = 0
    data['data'] = {}

    for a in range(4,5):
        print('Scraping page ',a)

        document = requests.get('https://indianexpress.com/section/cities/page/' + str(a))
        articles = bs(document.content, 'html.parser').select('div.articles')

        for article in articles:
            try:
                link = article.select_one('h2.title > a')['href']
                title = article.select_one('h2.title > a').text
                print(link,title)
                image = article.select_one('a > img')['data-lazy-srcset'].split(" ")[0]
                time = article.select_one('div.date').text.lstrip().replace('  ',' ')
                if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time, "%B %d, %Y %I:%M:%S %p")):
                    continue
                place_link = link.split('/')[5].capitalize()
                if(place_link in places):
                    addData(data, place_link, link, title, image)
                    continue
                for place in places:
                    if(re.search(place ,place_link,re.IGNORECASE)):
                        addData(data, place, link, title, image)
            except Exception as e:
                errors.append({data['source_tag']:link})
                    
    return data
print(inexp())

Scraping page  4
https://indianexpress.com/article/cities/mumbai/portraits-memorabilia-when-aapri-rani-elizabeth-lived-in-parsi-homes-memory-8159059/ Portraits, memorabilia: When ‘Aapri Rani’ Elizabeth lived in Parsi homes & memory
https://indianexpress.com/article/cities/kolkata/school-jobs-scam-cbi-questions-partha-again-8159105/ School jobs scam: CBI questions Partha again
https://indianexpress.com/article/cities/kolkata/nabanna-march-keep-highlighting-police-atrocities-during-stir-bansal-tells-bjp-workers-8159106/ Nabanna march: Keep highlighting ‘police atrocities’ during stir, Bansal tells BJP workers
https://indianexpress.com/article/cities/kolkata/cpim-workers-detained-before-start-of-rally-8159103/ CPI(M) workers detained before start of rally
https://indianexpress.com/article/cities/kolkata/4-held-for-titagarh-school-blast-police-say-three-of-them-alumni-8159095/ 4 held for Titagarh school blast, police say three of them alumni
https://indianexpress.com/article/cities/lucknow

TypeError: 'NoneType' object is not subscriptable

NDTV

In [ ]:
def ndtv():
    data = {}
    data['source_tag'] = 'ndtv'
    data['name'] = 'NDTV'
    data['image'] = 'https://cdn.ndtv.com/common/images/ogndtv.png'
    data['count'] = 0
    data['data'] = {}

    for a in range(1,15):
        print('Scraping page ',a)

        document = requests.get('https://www.ndtv.com/india/page-' + str(a))
        news_divs = bs(document.content, 'html.parser').select("div[class='news_Itm']")
        for news_item in news_divs:
            news_link = news_item.select_one('h2.newsHdng > a')
            image = news_item.select_one('div.news_Itm-img > a > img')['src']
            link = news_link['href']
            title = news_link.text
            doc = bs(requests.get(link).content,'html.parser')
            loc_item = doc.find("b", class_="place_cont")
            if doc.select_one("div.ins_instory_dv_cont > img")!= None:
                image = doc.select_one("div.ins_instory_dv_cont > img")['src']
            loc = None
            if(loc_item!=None):
                loc = loc_item.text[:-2]
            else:
                for place in places:
                    if(re.search(place ,link,re.IGNORECASE)):
                        loc = place
                        break
                if(loc==None):
                    continue
            tim = doc.find("span",attrs={"itemprop":"dateModified"})['content']
            if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(tim[:-9], "%Y-%m-%dT%H:%M")):
                data['data']['z'] = {'cordinates':[0,0], 'count':0, 'links':[]}
                return data

            if(loc in places):
                addData(data, loc, link, title, image)
                print(loc, title,end = "\n-------------------------------------\n")
                data['count']+=1
            
    data['data']['z'] = {'cordinates':[0,0], 'count':0, 'links':[]}
    return data
ndtv()


Republic World

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re
from geopy.geocoders import Nominatim
import datetime as dt
errors=[]
def rw():
    data = {}
    data['source_tag'] = 'rw'
    data['name'] = 'Republic World'
    data['image'] = 'https://bharat.republicworld.com/assets/images/rdot_icon_red.svg'
    data['count'] = 0 #excludes calibration
    data['data'] = {}

    for a in range(1,2):

        print('scraping page ',a )

        document = requests.get('https://www.republicworld.com/india-news/general-news/' + str(a))
        doc = bs(document.content, 'html.parser')
        for news in doc.select('article.hover-effect'):
            try:
                link = news.select_one('a')['href']
                image = news.select_one('img')['src']
                title = news.select_one('img')['title']
                time = bs(requests.get(link).content, 'html.parser').find('time')['datetime']

                if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time[:19],"%Y-%m-%dT%H:%M:%S")):
                    return data
                for place in places:
                    if(re.search("[\/-]"+place+"[\/-]",link,re.IGNORECASE)):
                        print(place,link)
                        addData(data, place,link, title, image)
                        data['count']+=1
            except Exception as e:
                errors.append((data['source_tag'],link,e))
    return data
rw()

India Today

In [12]:
from bs4 import BeautifulSoup as bs
import requests
import re
from geopy.geocoders import Nominatim
import datetime as dt

def idto():
    data = {}
    data['source_tag'] = 'idto'
    data['name'] = 'India Today'
    data['image'] = 'https://akm-img-a-in.tosshub.com/sites/indiatodaygroup/ITG-logo-main.png'
    data['count'] = 0 #excludes calibration
    data['data'] = {}
    domain = 'https://www.indiatoday.in'
    for a in range(0,10):
        page_data = json.loads(requests.get(f'https://www.indiatoday.in/api/ajax/loadmorecontent?page={a}&pagepath=/india').content)
        for article in page_data['data']['content']:
            try:
                title = article['title_short']
                image = article['big_story_image']
                link = domain + article['canonical_url']
                time = article['datetime_published']
                if(dt.datetime.now()-dt.timedelta(days=1)>dt.datetime.strptime(time[:19],"%Y-%m-%d %H:%M:%S")):
                    return data
                for place in places:
                    if(re.search("[\/-]"+place+"[\/-]",link,re.IGNORECASE)):
                        addData(data, place,link, title, image)
                        data['count']+=1
            except Exception as e:
                errors.append((data['source_tag'],link,e))   
    return data
idto()

0
1
2
3
4
5
6
7
8
9


{'source_tag': 'idto',
 'name': 'India Today',
 'image': 'https://akm-img-a-in.tosshub.com/sites/indiatodaygroup/ITG-logo-main.png',
 'count': 54,
 'data': {'Chennai': {'cordinates': [80.270186, 13.0836939],
   'count': 3,
   'links': [{'title': 'ISIS module busted in Chennai, 3 suspected of having links with terror outfit held',
     'url': 'https://www.indiatoday.in/india/video/isis-module-busted-in-chennai-3-suspected-of-having-links-with-terror-outfit-held-2296537-2022-11-12',
     'img': 'https://akm-img-a-in.tosshub.com/indiatoday/images/video/202211/chennai_police-sixteen_nine.jpeg?VersionId=ufX5SMYMSkE.DmHwq9uVyXP.V4BT6JeJ'},
    {'title': 'Corruption-free govt has led to India’s rapid economic growth: Amit Shah',
     'url': 'https://www.indiatoday.in/india/story/amit-shah-laud-govt-for-indian-economy-growth-gdp-chennai-event-2296466-2022-11-12',
     'img': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202211/pti11_12_2022_000182b-sixteen_nine.jpg?VersionId=oJlg34

In [24]:
bool("")

False

In [21]:
import json
import pandas as pd
json_inp = json.load(open('source-data.json'))
pl = {}
for news in json_inp:
    source = news['source_tag']
    for place in news['data']:
        if place not in pl:
            pl[place] = {}
            pl[place]['cordinates'] = news['data'][place]['cordinates']
            pl[place][source] = []
            pl[place][source].extend(news['data'][place]['links'])
        if place in pl:
            pl[place][source] = []
            pl[place][source].extend(news['data'][place]['links'])
with open('./place-data.json', 'w') as jso:
            json.dump(pl,jso,indent=3)

In [ ]:
import requests
import json
json_inp = requests.get('https://api.npoint.io/d45deb15252bacd419f4').json()
print(json_inp)